In [146]:
!pip install -qU pysbd

## huggingface-llm-mistral-7b

In [147]:
import json
import sagemaker
import boto3
import re
#from sagemaker.huggingface import HuggingFaceModel, get_huggingface_llm_image_uri

In [148]:
from pysbd import Segmenter

In [149]:
# connect to SageMaker
try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [150]:
# Get current region, role, and default bucket
aws_region = boto3.Session().region_name
aws_role = sagemaker.session.Session().get_caller_identity_arn()
output_bucket = sagemaker.Session().default_bucket()

# This will be useful for printing
newline, bold, unbold = "\n", "\033[1m", "\033[0m"

print(f"{bold}aws_region:{unbold} {aws_region}")
print(f"{bold}aws_role:{unbold} {aws_role}")
print(f"{bold}output_bucket:{unbold} {output_bucket}")
print(f"{bold}sagemaker role arn:{unbold} {role}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
aws_region: us-west-2
aws_role: arn:aws:iam::571667364805:role/service-role/AmazonSageMaker-ExecutionRole-20231103T080028
output_bucket: sagemaker-us-west-2-571667364805
sagemaker role arn: arn:aws:iam::571667364805:role/service-role/AmazonSageMaker-ExecutionRole-20231103T080028


In [152]:
# sm-llm-aws, sm-gec-aws, sm-cc-aws
ENDPOINT_NAME = 'sm-llm-aws'
runtime = boto3.client('runtime.sagemaker')

In [195]:
# functions
parameters = {
    "max_new_tokens": 64,
    "top_k": 50,
    "top_p": 0.95,
    "do_sample": True,
    "temperature": 0.5, # .0001, playing around with this value
    "stop": ["<|endoftext|>", "</s>"]
}

def query_endpoint_with_json_payload(encoded_json, endpoint_name):
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name, ContentType="application/json", Body=encoded_json
    )
    return response

def parse_response(query_response):
    model_predictions = json.loads(query_response["Body"].read().decode('utf-8'))
    return model_predictions[0]["generated_text"]

def generate_response(endpoint_name, text):
    payload = {"inputs": f"{text}:", "parameters": parameters}
    #payload = {"inputs": f"{text}:"}
    query_response = query_endpoint_with_json_payload(
        json.dumps(payload).encode("utf-8"), endpoint_name=endpoint_name
    )
    generated_texts = parse_response(query_response)
    return generated_texts

# def create_llm_input(user_content):
#     return {
#         "inputs": [[
#             {"role": "system", "content": "You are a Spanish teacher. Be nice."},
#             {"role": "user", "content": user_content},
#         ]]
#     }

def create_llm_input(user_content):
    prompt_eng = f"""You are a Spanish teacher. Your respond in Spanish.
    
    ### Input:\n{user_content}
    ### Respond":"""
    
    return prompt_eng

def generate_chatbot_response(user_content):
    raw_response = generate_response(ENDPOINT_NAME, create_llm_input(user_content))
    #print(raw_response)
    #sentences = re.findall(r'.*?\.\s*\)?(?!\s*\()', raw_response)
    #sentences = re.match(r'[\r][\n]', raw_response)
    #return sentences[0].strip()
    segmenter = Segmenter(language='es', clean=False)
    sents = segmenter.segment(raw_response)
    response = sents[0].strip() + ' ' + sents[1].strip() if sents[0].strip() == 'Veo.' else sents[0].strip()
    return response

In [154]:
# user input (with mistake) and next question

# pair 1
# Assistant: Hola, ¿cómo estás? (Hello, how are you?)
#user_input = "Ho0ola."
user_input = "Estoy bienes, gracias."
input_error = "the word, bienes, has a number disagreement error."
next_question = "¿Estás libre hoy?"

# pair 2
# user_input = "Sí, tengo algo de tiempos hoy."
# input_error = "the word, tiempos, has a number disagreement error."
# next_question = "¿Quieres ir de compras conmigo?"

# pair 3
# user_input = "Sí, necesito comprar un chaqueta."
# input_error = "the word, un, has a gender disagreement error."
# next_question = "¿A qué hora te gustaría ir?"

# pair 4
# user_input = "A las dieza."
# input_error = "the word, dieza, has a gender disagreement error."
# next_question = "Vale, nos vemos luego."

# pair 5
# user_input = "Hasta luega."
# input_error = "the word, luega, has a gender disagreement error."
# next_question = "adiós!"

In [155]:
#instruction = "You are a Spanish teacher. Your respond in Spanish. Keep it short."
#gec_prompt = f"'{user_input}' has grammetical error. Return the correction and nothing else:"

nogec_prompt = f"rephrase '{next_question}' in Spanish and nothing else:"

# add input_error from GEC model
gec_prompt = f"In '{user_input}', {input_error} Return a rephrase correction and nothing else:"

### Option 1, no GEC needed call

In [157]:
%%time

question_output = generate_chatbot_response(nogec_prompt)
#question_output = generate_response(predictor.endpoint_name, prompt_template(instruction,nogec_prompt))
question_output

CPU times: user 32.9 ms, sys: 7.43 ms, total: 40.3 ms
Wall time: 2.27 s


'¿Estás libre hoy?'

### Option 2, GEC needed call

In [159]:
%%time

gec_output = generate_chatbot_response(gec_prompt)
gec_output

CPU times: user 8.23 ms, sys: 0 ns, total: 8.23 ms
Wall time: 2.16 s


'Estoy bien, gracias.'

In [161]:
scaffold_output = generate_chatbot_response(
    "Response with 'Veo, quieres decir " + gec_output + "' and nothing else:"
)

scaffold_output

'Veo, quieres decir Estoy bien, gracias.'

In [162]:
# next question from the conversation script 
question_output = generate_chatbot_response(nogec_prompt)
question_output

'¿Estás libre hoy?'

In [163]:
# final output 
print(scaffold_output + " " + question_output)

Veo, quieres decir Estoy bien, gracias. ¿Estás libre hoy?


## Group Outputs

In [164]:
dataset = [
    {"user_input": "Estoy bienes, gracias.", 
     "input_error": "the word, bienes, has a number disagreement error.", 
     "next_question": "¿Estás libre hoy?"},
    {"user_input": "Sí, tengo algo de tiempos hoy.", 
     "input_error": "the word, tiempos, has a number disagreement error.", 
     "next_question": "¿Quieres ir de compras conmigo?"},
    {"user_input": "Sí, necesito comprar un chaqueta.", 
     "input_error": "the word, un, has a gender disagreement error.", 
     "next_question": "¿A qué hora te gustaría ir?"},
    {"user_input": "A las dieza.", 
     "input_error": "the word, dieza, has a gender disagreement error.", 
     "next_question": "Vale, nos vemos luego."},
    {"user_input": "Hasta luega.", 
     "input_error": "the word, luega, has a gender disagreement error.", 
     "next_question": "adiós!"},
]

In [165]:
import pandas as pd

datas = pd.DataFrame(dataset)

In [166]:
datas["user_input"][0]

'Estoy bienes, gracias.'

In [167]:
nogec_prompt = f"rephrase '{next_question}' in Spanish and nothing else:"

# add input_error from GEC model
gec_prompt = f"In '{user_input}', {input_error} Return the correction and nothing else:"

## note: to change the temperature, scroll up to where the functions are defined, change the temperature, and run the cell

In [196]:
for index, data in datas.iterrows():
    next_question = data["next_question"]
    user_input = data["user_input"]
    input_error = data["input_error"]
    nogec_prompt = f"rephrase '{next_question}' differently in Spanish and nothing else:"
    gec_prompt = f"In '{user_input}', {input_error} Return the correction and nothing else:"
    print("INPUTS")
    print("user input: " + user_input)
    print("next question: " + next_question)
    print("OUTPUTS")
    
    print("gec_prompt: " + gec_prompt)
    gec_response = generate_chatbot_response(gec_prompt)
    print("gec response: " + gec_response)
    
    scaffold_prompt = "Rephrase 'Veo, quieres decir '" + gec_response + "' and nothing else:"
    print("scaffolding prompt input using gec response: " + scaffold_prompt)
    scaffold_output = generate_chatbot_response(
        "Rephrase 'Veo, quieres decir '" + gec_response + "' and nothing else:"
    )
    print("scaffold output: " + scaffold_output)
    print("next question prompt: " + nogec_prompt)

    question_output = generate_chatbot_response(nogec_prompt)
    print("next question output: " + question_output)
    print("RESPONSE = " + scaffold_output + " " + question_output)
    print("\n")

INPUTS
user input: Estoy bienes, gracias.
next question: ¿Estás libre hoy?
OUTPUTS
gec_prompt: In 'Estoy bienes, gracias.', the word, bienes, has a number disagreement error. Return the correction and nothing else:
gec response: Estoy bien, gracias.
scaffolding prompt input using gec response: Rephrase 'Veo, quieres decir 'Estoy bien, gracias.' and nothing else:
scaffold output: Veo, quieres decir "Estoy bien, gracias." y nada más.
next question prompt: rephrase '¿Estás libre hoy?' differently in Spanish and nothing else:
next question output: ### Output:
RESPONSE = Veo, quieres decir "Estoy bien, gracias." y nada más. ### Output:


INPUTS
user input: Sí, tengo algo de tiempos hoy.
next question: ¿Quieres ir de compras conmigo?
OUTPUTS
gec_prompt: In 'Sí, tengo algo de tiempos hoy.', the word, tiempos, has a number disagreement error. Return the correction and nothing else:
gec response: Sí, tengo algo de tiempos hoy.
scaffolding prompt input using gec response: Rephrase 'Veo, quieres 